In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE57162'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A genomic algorithm for the molecular classification of renal cortical neoplasms: Development and Validation"
!Series_summary	"Accurate diagnostic discrimination of benign renal oncocytoma (OC) and malignant renal cell carcinomas (RCC) is not only useful for planning appropriate treatment strategies of patients with renal masses but also for estimating prognosis. Classification of renal neoplasms solely by histopathology can often be challenging for a variety of reasons. The aim of this study was to develop and validate a genomic algorithm for molecular classification of renal cortical neoplasms that could be implemented in a routine clinical diagnostic setting. Using TCGA (The Cancer Genome Atlas) copy number profiles of over 600 RCC specimens, prior FISH studies and published literature, a classification algorithm was developed consisting of 15 genomic markers: loss of VHL, 3p21, 8p, and chromosomes 1, 2, 6, 10 and 17, and gain of 5qter, 16p, 17

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None
convert_trait = convert_age = convert_gender = None

# Determine gene expression data availability
is_gene_available = True  # This dataset is assumed to be suitable

# Determine the availability and keys for the variables
trait_row = 2  # histology seems to cover cancer type, including pRCC
age_row = 4  # age values are listed under key 4
gender_row = 3  # gender values are listed under key 3

# Define conversion functions
def convert_trait(value):
    if 'pRCC' in value:
        return 1
    else:
        return 0

def convert_age(value):
    try:
        return int(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender_val = value.split(': ')[1]
    if gender_val == 'M':
        return 1
    elif gender_val == 'F':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE57162', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical features if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE57162.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1376393': [0, 66, 0], 'GSM1376394': [0, 82, 0], 'GSM1376395': [0, 59, 0], 'GSM1376396': [0, 58, 0], 'GSM1376397': [0, 59, 0], 'GSM1376398': [0, 57, 0], 'GSM1376399': [0, 62, 0], 'GSM1376400': [0, 48, 0], 'GSM1376401': [0, 69, 0], 'GSM1376402': [0, 76, 0], 'GSM1376403': [0, 63, 0], 'GSM1376404': [0, 74, 0], 'GSM1376405': [0, 57, 0], 'GSM1376406': [0, 77, 0], 'GSM1376407': [1, 79, 0], 'GSM1376408': [0, 45, 0], 'GSM1376409': [1, 67, 0], 'GSM1376410': [0, 57, 0], 'GSM1376411': [0, 59, 0], 'GSM1376412': [0, 69, 0], 'GSM1376413': [0, 52, 0], 'GSM1376414': [0, 68, 0], 'GSM1376415': [0, 43, 0], 'GSM1376416': [0, 39, 0], 'GSM1376417': [0, 53, 0], 'GSM1376418': [0, 49, 0], 'GSM1376419': [0, 47, 0], 'GSM1376420': [0, 56, 0], 'GSM1376421': [0, 61, 0], 'GSM1376422': [1, 72, 0], 'GSM1376423': [0, 66, 0], 'GSM1376424': [1, 41, 0], 'GSM1376425': [1, 67, 0], 'GSM1376426': [0, 58, 0], 'GSM1376427': [0, 55, 0], 'GSM1376428': [0, 65, 0], 'GSM1376429': [0, 58, 0], 'GSM1376430': [0, 41, 0], 'GSM1376431

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'SPOT_ID': ['HsCGHBrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Read the dictionary and decide which key stores the same kind of identifiers as in STEP3, and which key stores the gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE57162.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# 4. Save the cohort information.
save_cohort_info('GSE57162', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE57162.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Kidney_Papillary_Cell_Carcinoma', the least common label is '1.0' with 57 occurrences. This represents 29.84% of the dataset.
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 54.0
  50% (Median): 62.0
  75%: 69.0
Min: 27.0
Max: 88.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 63 occurrences. This represents 32.98% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

